# Sessions & Preferences Demo

**Demonstrating how to use sessions and user preferences for personalized experiences.**

This notebook shows practical examples of:
1. Creating sessions with user preferences
2. Preferences persisting across sessions (user-scoped state)
3. Querying session history from database
4. Multi-user support
5. Context management for long conversations

## Setup

In [ ]:
import os
import uuid
import json
import sqlite3
from dotenv import load_dotenv

from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner

from services.session_service import create_session_service
from services.context_manager import ContextManager
from tools.google_books import google_books_tool
from agents.orchestrator import create_workflow

load_dotenv()

print("✅ Imports complete")
print(f"   Google API Key: {'OK' if os.getenv('GOOGLE_API_KEY') else 'MISSING'}")

## Scenario 1: First-Time User with Preferences

Create a new user session with travel preferences that will persist.

In [ ]:
# Initialize services with database
session_service = create_session_service(use_database=True)

# Configure model
retry_config = types.HttpRetryOptions(
    attempts=5, exp_base=7, initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

model = Gemini(
    model="gemini-2.0-flash-lite",
    api_key=os.getenv('GOOGLE_API_KEY'),
    retry_options=retry_config
)

print("✅ Services initialized")

In [ ]:
# Create first session with user preferences
session_1_id = str(uuid.uuid4())
user_id = "alice"

# Define user preferences
user_preferences = {
    "prefers_museums": True,
    "travels_with_kids": False,
    "budget": "moderate",
    "favorite_genres": ["historical fiction", "classics"],
    "dietary_restrictions": ["vegetarian"],
    "preferred_pace": "relaxed"
}

await session_service.create_session(
    app_name="storyland",
    user_id=user_id,
    session_id=session_1_id,
    state={
        "book_title": "Pride and Prejudice",
        "author": "Jane Austen",
        "user:preferences": user_preferences  # User-scoped!
    }
)

print(f"✅ Session 1 created for user '{user_id}'")
print(f"   Session ID: {session_1_id[:8]}...")
print(f"   Book: Pride and Prejudice")
print(f"   Preferences saved:")
print(f"     - Museums: {user_preferences['prefers_museums']}")
print(f"     - Budget: {user_preferences['budget']}")
print(f"     - Pace: {user_preferences['preferred_pace']}")

## Scenario 2: Returning User - Preferences Auto-Loaded

When the same user creates a new session, their preferences automatically carry over via the `user:` prefix.

In [ ]:
# Create second session for same user
session_2_id = str(uuid.uuid4())

await session_service.create_session(
    app_name="storyland",
    user_id=user_id,  # Same user!
    session_id=session_2_id,
    state={
        "book_title": "Gone with the Wind",
        "author": "Margaret Mitchell"
        # Notice: No preferences specified!
    }
)

# Get the session and check preferences
session_2 = await session_service.get_session(
    app_name="storyland",
    user_id=user_id,
    session_id=session_2_id
)

# Check if preferences are there
loaded_preferences = session_2.state.get("user:preferences")

print(f"✅ Session 2 created for user '{user_id}'")
print(f"   Session ID: {session_2_id[:8]}...")
print(f"   Book: Gone with the Wind")
print(f"\n🎯 Preferences automatically loaded from previous session:")

if loaded_preferences:
    print(json.dumps(loaded_preferences, indent=2))
else:
    print("   ⚠️  No preferences found (check database persistence)")

## Scenario 3: Resume a Previous Session

Continue working on an existing itinerary.

In [ ]:
# Load session 1 again
resumed_session = await session_service.get_session(
    app_name="storyland",
    user_id=user_id,
    session_id=session_1_id  # Original session ID
)

print(f"📂 Resumed session: {session_1_id[:8]}...")
print(f"\n   Book: {resumed_session.state.get('book_title')}")
print(f"   Author: {resumed_session.state.get('author')}")

# Check if itinerary exists
itinerary = resumed_session.state.get("final_itinerary")
if itinerary:
    print(f"\n   ✅ Previous itinerary found:")
    print(f"      Cities: {len(itinerary.get('cities', []))}")
else:
    print(f"\n   ⚠️  No itinerary found in this session (not yet generated)")

print(f"\n   Number of events in conversation: {len(resumed_session.events)}")

## Scenario 4: Query Database Directly

Use SQL to analyze user behavior and history.

In [ ]:
def get_user_history(user_id: str):
    """Get all books a user has explored."""
    conn = sqlite3.connect('storyland_sessions.db')
    cursor = conn.cursor()
    
    # Query sessions for this user
    cursor.execute("""
        SELECT 
            id,
            json_extract(state, '$.book_title') as book_title,
            json_extract(state, '$.author') as author,
            create_time
        FROM sessions
        WHERE user_id = ?
        ORDER BY create_time DESC
    """, (user_id,))
    
    results = cursor.fetchall()
    conn.close()
    return results

# Get Alice's history
history = get_user_history(user_id)

print(f"📚 User '{user_id}' has explored {len(history)} books:")
print()

for i, (sess_id, book, author, created) in enumerate(history, 1):
    print(f"   {i}. {book or 'Unknown'} by {author or 'Unknown'}")
    print(f"      Session: {sess_id[:8]}...")
    print(f"      Created: {created}")
    print()

## Scenario 5: Build User Profile Over Time

Analyze preferences and behavior to create a rich user profile.

In [ ]:
def get_user_profile(user_id: str):
    """Build a user profile from database."""
    conn = sqlite3.connect('storyland_sessions.db')
    cursor = conn.cursor()
    
    # Get all sessions
    cursor.execute("""
        SELECT 
            COUNT(*) as total_sessions,
            MIN(create_time) as first_session,
            MAX(create_time) as last_session
        FROM sessions
        WHERE user_id = ?
    """, (user_id,))
    
    stats = cursor.fetchone()
    
    # Get preferences from most recent session
    cursor.execute("""
        SELECT json_extract(state, '$."user:preferences"') as preferences
        FROM sessions
        WHERE user_id = ?
        ORDER BY create_time DESC
        LIMIT 1
    """, (user_id,))
    
    prefs_row = cursor.fetchone()
    preferences = json.loads(prefs_row[0]) if prefs_row and prefs_row[0] else {}
    
    # Get all books explored
    cursor.execute("""
        SELECT json_extract(state, '$.book_title') as book
        FROM sessions
        WHERE user_id = ?
        ORDER BY create_time
    """, (user_id,))
    
    books = [row[0] for row in cursor.fetchall() if row[0]]
    
    conn.close()
    
    return {
        "user_id": user_id,
        "total_sessions": stats[0],
        "first_visit": stats[1],
        "last_visit": stats[2],
        "preferences": preferences,
        "books_explored": books
    }

# Build profile
profile = get_user_profile(user_id)

print(f"👤 USER PROFILE: {profile['user_id']}")
print("=" * 60)
print(f"\n📊 Activity:")
print(f"   Total sessions: {profile['total_sessions']}")
print(f"   First visit: {profile['first_visit']}")
print(f"   Last visit: {profile['last_visit']}")

print(f"\n📚 Books Explored ({len(profile['books_explored'])})::")
for book in profile['books_explored']:
    print(f"   - {book}")

print(f"\n⚙️  Preferences:")
if profile['preferences']:
    for key, value in profile['preferences'].items():
        print(f"   - {key}: {value}")
else:
    print("   No preferences set")

## Scenario 6: Update User Preferences

User preferences can be updated in new sessions.

In [ ]:
# User decides they now travel with kids
session_3_id = str(uuid.uuid4())

# Update preferences
updated_preferences = user_preferences.copy()
updated_preferences["travels_with_kids"] = True
updated_preferences["preferred_pace"] = "moderate"  # Changed from relaxed

await session_service.create_session(
    app_name="storyland",
    user_id=user_id,
    session_id=session_3_id,
    state={
        "book_title": "Harry Potter and the Philosopher's Stone",
        "author": "J.K. Rowling",
        "user:preferences": updated_preferences  # Updated!
    }
)

print(f"✅ Session 3 created with updated preferences")
print(f"\n   Changes:")
print(f"     travels_with_kids: False → True")
print(f"     preferred_pace: relaxed → moderate")
print(f"\n   These new preferences will be used in future sessions!")

## Scenario 7: Multi-User Support

Different users have isolated sessions and preferences.

In [ ]:
# Create session for a different user
bob_session_id = str(uuid.uuid4())
bob_user_id = "bob"

bob_preferences = {
    "prefers_museums": False,
    "budget": "luxury",
    "favorite_genres": ["science fiction"],
    "preferred_pace": "fast-paced"
}

await session_service.create_session(
    app_name="storyland",
    user_id=bob_user_id,
    session_id=bob_session_id,
    state={
        "book_title": "Dune",
        "author": "Frank Herbert",
        "user:preferences": bob_preferences
    }
)

print(f"✅ Created session for user '{bob_user_id}'")
print(f"\n   Alice's preferences:")
print(f"     - Museums: {user_preferences['prefers_museums']}")
print(f"     - Budget: {user_preferences['budget']}")
print(f"     - Pace: {user_preferences['preferred_pace']}")

print(f"\n   Bob's preferences (completely different):")
print(f"     - Museums: {bob_preferences['prefers_museums']}")
print(f"     - Budget: {bob_preferences['budget']}")
print(f"     - Pace: {bob_preferences['preferred_pace']}")

print(f"\n   Each user's data is completely isolated!")

## Scenario 8: Context Manager - Tracking Conversation Stats

Use the Context Manager to monitor conversation size and token usage.

In [ ]:
# Create context manager with default settings
context_manager = ContextManager(max_events=20, max_tokens=8000)

print("📊 CONTEXT MANAGER")
print("=" * 60)
print(f"   Max events: {context_manager.max_events}")
print(f"   Max tokens: {context_manager.max_tokens}")
print(f"   Preserve system prompts: {context_manager.preserve_system}")

# Create mock events to simulate conversation
from google.genai import types

class MockEvent:
    def __init__(self, text):
        self.content = types.Content(parts=[types.Part(text=text)])

mock_events = [
    MockEvent("User: Tell me about Pride and Prejudice"),
    MockEvent("Assistant: Pride and Prejudice is a novel by Jane Austen..."),
    MockEvent("User: What places can I visit related to this book?"),
    MockEvent("Assistant: You can visit Bath, England where Jane Austen lived..."),
    MockEvent("User: Create an itinerary for 3 days"),
    MockEvent("Assistant: Here's a 3-day literary tour of Jane Austen's England..."),
]

# Get context statistics
stats = context_manager.get_context_stats(mock_events)

print(f"\n📈 Context Statistics:")
print(f"   Number of events: {stats['num_events']}")
print(f"   Total characters: {stats['total_chars']}")
print(f"   Estimated tokens: {stats['estimated_tokens']}")
print(f"   Within event limit: {stats['within_limit']}")

## Scenario 9: Context Manager - Sliding Window Compaction

When conversations grow too long, use sliding window to keep only recent events.

In [ ]:
# Create a longer conversation that exceeds limits
long_conversation = [MockEvent(f"Message {i}: " + "x" * 200) for i in range(25)]

print(f"📦 SLIDING WINDOW COMPACTION")
print("=" * 60)
print(f"\n   Original conversation: {len(long_conversation)} events")

# Check if compaction is needed
needs_compaction = context_manager.should_compact(long_conversation)
print(f"   Needs compaction: {needs_compaction}")

if needs_compaction:
    # Apply sliding window
    compacted = context_manager.limit_events(long_conversation, num_recent=15)
    
    print(f"\n   After compaction: {len(compacted)} events")
    print(f"   Events removed: {len(long_conversation) - len(compacted)}")
    
    # Show stats before and after
    before_stats = context_manager.get_context_stats(long_conversation)
    after_stats = context_manager.get_context_stats(compacted)
    
    print(f"\n   📊 Token comparison:")
    print(f"      Before: ~{before_stats['estimated_tokens']} tokens")
    print(f"      After:  ~{after_stats['estimated_tokens']} tokens")
    print(f"      Saved:  ~{before_stats['estimated_tokens'] - after_stats['estimated_tokens']} tokens")

## Scenario 10: Inspect Database

Check what's actually in the database.

In [ ]:
# Get database statistics
conn = sqlite3.connect('storyland_sessions.db')
cursor = conn.cursor()

# Total sessions
cursor.execute("SELECT COUNT(*) FROM sessions")
total_sessions = cursor.fetchone()[0]

# Unique users
cursor.execute("SELECT COUNT(DISTINCT user_id) FROM sessions")
unique_users = cursor.fetchone()[0]

# Sessions per user
cursor.execute("""
    SELECT user_id, COUNT(*) as session_count
    FROM sessions
    GROUP BY user_id
    ORDER BY session_count DESC
""")
user_stats = cursor.fetchall()

conn.close()

print(f"📊 DATABASE STATISTICS")
print("=" * 60)
print(f"\n   Total sessions: {total_sessions}")
print(f"   Unique users: {unique_users}")
print(f"\n   Sessions per user:")
for user, count in user_stats:
    print(f"     - {user}: {count} sessions")

## Scenario 11: Full Workflow with Preferences

Create a workflow that uses user preferences for personalization.

In [ ]:
# Create workflow - reader_profile_agent is always included
workflow = create_workflow(model, google_books_tool)

print("🔧 WORKFLOW STRUCTURE")
print("=" * 60)

print(f"\n   Sub-agents: {len(workflow.sub_agents)}")
for i, agent in enumerate(workflow.sub_agents, 1):
    print(f"      {i}. {agent.name}")

print(f"\n   ✨ reader_profile_agent reads user:preferences from session state")
print(f"   ✨ trip_composer creates personalized itinerary based on preferences")

## Summary

This notebook demonstrated:

✅ **Scenario 1**: Creating sessions with user preferences  
✅ **Scenario 2**: Automatic preference loading for returning users  
✅ **Scenario 3**: Resuming previous sessions  
✅ **Scenario 4**: Direct SQL queries for user history  
✅ **Scenario 5**: Building comprehensive user profiles  
✅ **Scenario 6**: Updating user preferences over time  
✅ **Scenario 7**: Multi-user isolation and support  
✅ **Scenario 8**: Context Manager for tracking conversation stats  
✅ **Scenario 9**: Sliding window compaction for long conversations  
✅ **Scenario 10**: Database inspection and statistics  
✅ **Scenario 11**: Full workflow with personalization  

### Key Takeaways

1. **User-scoped state** (`user:preferences`) persists across sessions automatically
2. **Session state** is specific to each conversation
3. **Database persistence** enables analytics and user profiling
4. **Context Manager** keeps conversations within token budgets
5. **Multi-user support** keeps data isolated and secure
6. **Preferences flow**: Session state → reader_profile_agent → trip_composer

### CLI Usage

```bash
# Basic usage
python main.py "Pride and Prejudice"

# With database persistence
python main.py "Pride and Prejudice" --database

# With preferences
python main.py "Pride and Prejudice" --budget luxury --pace relaxed --museums

# Family trip
python main.py "Harry Potter" --with-kids --budget moderate
```

## Cleanup (Optional)

Remove test data if needed.

In [ ]:
# Uncomment to delete test sessions
# import os
# if os.path.exists('storyland_sessions.db'):
#     os.remove('storyland_sessions.db')
#     print("✅ Database deleted")

print("Run the cell above to clean up test data")